In [6]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Get the database connection details from the environment
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

# Create the connection string
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create the engine
engine = create_engine(connection_string)

# Example query (modify as needed)
df_engagement = pd.read_sql("SELECT * FROM user_engagement", engine)
df_experience = pd.read_sql("SELECT * FROM xdr_data", engine)

# Print the dataframes to confirm successful connection
print(df_engagement.head())
print(df_experience.head())


        user_id  session_frequency  session_duration  total_traffic
0  2.082018e+14                  2          140845.0   1.276832e+09
1  2.082014e+14                  1           85950.0   8.926375e+08
2           NaN                  9        37507441.0   2.815223e+11
3  2.082014e+14                  1           26612.0   3.209849e+08
4  2.082003e+14                  1           51265.0   5.779483e+08
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.57

In [7]:
# Ensure data is loaded before proceeding
if df_engagement is not None and df_experience is not None:
    # Merge datasets on 'user_id' (make sure this key is available in both tables)
    df_combined = pd.merge(df_engagement, df_experience, left_on='user_id', right_on='IMSI', how='inner')

    # Calculate engagement and experience scores
    df_combined['engagement_score'] = df_combined[['session_frequency', 'session_duration', 'total_traffic']].mean(axis=1)
    df_combined['experience_score'] = df_combined[['Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']].mean(axis=1)

    # Handle missing values (Option 1: Drop rows with missing values)
    df_combined = df_combined.dropna(subset=['engagement_score', 'experience_score'])

    # Perform KMeans clustering
    X = df_combined[['engagement_score', 'experience_score']].values
    kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

    # Assign cluster labels
    df_combined['cluster'] = kmeans.labels_

    # Cluster summary
    cluster_summary = df_combined.groupby('cluster')[['engagement_score', 'experience_score']].mean()
    print("Cluster Summary:")
    print(cluster_summary)

    # Optionally, save the resulting dataframe back to your database
    df_combined.to_sql('clustered_user_data', engine, if_exists='replace', index=False)

else:
    print("Failed to load data.")

Cluster Summary:
         engagement_score  experience_score
cluster                                    
0            3.068169e+08       5059.386922
1            9.385327e+10       4388.218805
